- 模型应用
  - 模型加载
  - 模型测试集评估
    - 准确率
    - 召回率
    - F1


In [16]:
# 测试集评论数据测试

import time

import torch
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

import dataset
import dictionary
import model
from init import config

stopwords = dataset.load_stopwords(config.stopwords_path)
model = model.SentimentAnalysisModel().load(config.model_pt_path).to(config.device)
print('test_model[{:s}]'.format(config.model_pt_path), )
dictionary = dictionary.SentimentAnalysisDictionary().load(config.dictionary_pt_path)
print('dictionary[{:s}]'.format(config.dictionary_pt_path), )
test_contents, test_emotions = dataset.load_contents_labels(config.test_data_path)
test_tokens = dataset.tokenization(test_contents, stopwords)
test_dataset = dataset.SentimentAnalysisDataset(tokens=test_tokens, emotions=test_emotions,
                                                sequence_length=config.model_sequence_length,
                                                dictionary=dictionary, ).dataset

loss_fn = torch.nn.CrossEntropyLoss()
model.eval()
data, targets = test_dataset.tensors[0].to(config.device), test_dataset.tensors[1].to(config.device)
output = model(data)
test_loss = loss_fn(output, targets).item()
predicted = output.argmax(dim=1).cpu().numpy()
labels = targets.cpu().numpy()
print("每个类别的精确率和召回率：\n",
      classification_report(labels, predicted, target_names=dataset.emotion2label.keys()))

test_macro_precision = precision_score(labels, predicted, average='macro', zero_division=0)  # 宏精准率
test_macro_recall = recall_score(labels, predicted, average='macro', zero_division=0)  # 宏召回率
test_macro_f1 = f1_score(labels, predicted, average='macro')  # 宏F1值
print('test_loss[{:f}]'.format(test_loss),
      'test_macro_precision[{:f}]'.format(test_macro_precision),
      'test_macro_recall[{:f}]'.format(test_macro_recall),
      'test_macro_f1[{:f}]'.format(test_macro_f1),
      time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))

test_model[pt/sentiment_analysis_GRU_512_0.2_ReLU_100_24_128_1_0.001_0.8_pt]
dictionary[pt/dictionary.pt]
每个类别的精确率和召回率：
               precision    recall  f1-score   support

        非常满意       0.57      0.46      0.51       885
          满意       0.54      0.29      0.38      1247
          中立       0.13      0.45      0.21       272
         不满意       0.41      0.89      0.56      1150
       非常不满意       0.71      0.02      0.04      1320
        无效评论       0.31      0.83      0.45        42

    accuracy                           0.40      4916
   macro avg       0.44      0.49      0.36      4916
weighted avg       0.53      0.40      0.34      4916

test_loss[1.343639] test_macro_precision[0.444258] test_macro_recall[0.489539] test_macro_f1[0.356997] 2024-10-28 13:53:48


In [13]:
import numpy
# 随机评论数据测试

import model
import dataset
import dictionary
from init import config

stopwords = dataset.load_stopwords(config.stopwords_path)
model = model.SentimentAnalysisModel().load(config.model_pt_path).to(config.device)
print('test_model[{:s}]'.format(config.model_pt_path), )
dictionary = dictionary.SentimentAnalysisDictionary().load(config.dictionary_pt_path)
print('dictionary[{:s}]'.format(config.dictionary_pt_path), )

contents = ["感觉这个秤非常小巧精致实用。自从看到家人买来这个秤以后，其他的包括秤（包括金器秤）都被闲置了",
            "第三次购买了，还算可以，希望能更耐用一些。",
            "没什想说的，中规中矩，没什么色差",
            "秤盘不平，一边高一边低。这个价位不应该这样了。按键硬，按起来费力。底座盘不平，有一个脚悬空",
            "底座四个点不平，质量不行，本来就是有精度要求，结果买一个底座不平的电子秤。简直是一个笑话",
            "182712798172498172948718924791872489172894817"]

tokens = dataset.tokenization(contents, stopwords)
tokens_truncated = [token[:config.model_sequence_length] for token in tokens]
tokens_padded = [dataset.padding(token, config.model_sequence_length) for token in tokens_truncated]
tokens_ids = [[dictionary.word_id_dict[word] for word in token] for token in tokens_padded]
for i, token_id in enumerate(tokens_ids):
    data = torch.LongTensor(numpy.array(token_id)).unsqueeze(0)
    output = model(data)
    predicted = output.argmax(dim=1).cpu().numpy()
    print(contents[i], predicted[0], dataset.label2emotion.get(predicted[0]))

test_model[pt/sentiment_analysis_GRU_512_0.2_ReLU_100_24_128_1_0.001_0.8_pt]
dictionary[pt/dictionary.pt]
感觉这个秤非常小巧精致实用。自从看到家人买来这个秤以后，其他的包括秤（包括金器秤）都被闲置了 0 非常满意
第三次购买了，还算可以，希望能更耐用一些。 4 非常不满意
没什想说的，中规中矩，没什么色差 4 非常不满意
秤盘不平，一边高一边低。这个价位不应该这样了。按键硬，按起来费力。底座盘不平，有一个脚悬空 0 非常满意
底座四个点不平，质量不行，本来就是有精度要求，结果买一个底座不平的电子秤。简直是一个笑话 0 非常满意
182712798172498172948718924791872489172894817 5 无效评论
